<a href="https://colab.research.google.com/github/JHyunjun/SNU/blob/main/FFM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [39]:
import pandas as pd
import numpy as np
import torch
from torch import nn

In [40]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [41]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/snu/w4/실습자료/7-28 ffm-adult/data/adult.data', header=None)
df.head()
# Income = Column 14(Label)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [42]:
#for data pre-processing, only use String type column
from sklearn.preprocessing import LabelEncoder

df_out = df.copy()
df_out.drop(columns=[0, 2, 4, 10, 11, 12], inplace=True) #inplace = True : data frame에 직접적용하겠다. False : dataframe은 건드리지 않고 새로운 data frame을 만들어서 return 하겠다.(새로운 변수에)
num_values = []
for col in df_out.columns:
    df_out[col] = LabelEncoder().fit_transform(df_out[col]) #String Data를 Numerical하게 바꾸고 다시 원래자리에 저장하겠다 (LabelEncoder)
    num_values.append(df_out[col].max() + 1)
    print("col : ", col, "df_out[col].max() : ",df_out[col].max())

print(num_values)
print(len(num_values))


col :  1 df_out[col].max() :  8
col :  3 df_out[col].max() :  15
col :  5 df_out[col].max() :  6
col :  6 df_out[col].max() :  14
col :  7 df_out[col].max() :  5
col :  8 df_out[col].max() :  4
col :  9 df_out[col].max() :  1
col :  13 df_out[col].max() :  41
col :  14 df_out[col].max() :  1
[9, 16, 7, 15, 6, 5, 2, 42, 2]
9


In [43]:
print(df_out)

       1   3   5   6   7   8   9   13  14
0       7   9   4   1   1   4   1  39   0
1       6   9   2   4   0   4   1  39   0
2       4  11   0   6   1   4   1  39   0
3       4   1   2   6   0   2   1  39   0
4       4   9   2  10   5   2   0   5   0
...    ..  ..  ..  ..  ..  ..  ..  ..  ..
32556   4   7   2  13   5   4   0  39   0
32557   4  11   2   7   0   4   1  39   1
32558   4  11   6   1   4   4   0  39   0
32559   4  11   4   1   3   4   1  39   0
32560   5  11   2   4   5   4   0  39   1

[32561 rows x 9 columns]


In [44]:
data_x = df_out.iloc[:, :-1].values #train
data_y = df_out.iloc[:, -1].values #test

# 90% Train/10% Test Randomly
mask  = np.random.rand(df.shape[0]) < 0.9
trn_x = data_x[mask]
trn_y = data_y[mask]
test_x = data_x[~mask]
test_y = data_y[~mask]

print(trn_x.shape, test_x.shape)
print(trn_y.shape, test_y.shape)

(29338, 8) (3223, 8)
(29338,) (3223,)


In [45]:
from torch.utils.data import DataLoader, TensorDataset

batch_size = 128
trn_data = TensorDataset(torch.from_numpy(trn_x), torch.from_numpy(trn_y))
trn_loader = DataLoader(trn_data, batch_size, shuffle=True)
test_data = TensorDataset(torch.from_numpy(test_x), torch.from_numpy(test_y))
test_loader = DataLoader(test_data, batch_size)

In [46]:
class LinearModel(nn.Module) : 
  def __init__(self, num_values) : 
    super().__init__()
    weights = []
    for n in num_values : 
      weights.append(nn.Embedding(n,1)) #각 column별 전체 값들을 1개의 scalar로 매핑
      #num_values는 column마다 최대값인데 0부터 값이있다. 즉, feature의 갯수라고 볼 수있음
    self.weights = nn.ModuleList(weights) #ModuleList에 넣으면 Vector의 size가 다른것들도 하나로 묶어줄 수 있음.
    #  w1 [num_values[1],1]
    #  w2
    #  w3
    #  이런형태

  def forward(self,x) : 
    w_out = []
    for i in range(x.size(1)) : #num_values와 같은 size = x.size(1), Field Index
      #x[:,i] : x1, x2, x3, ... num_values만큼
      w_out.append(self.weights[i](x[:, i])) #Matrix multiplication
      # w_out 형태 = [ [batch,1] [batch,1] ... num_values갯수만큼]
      # [ [w1x1] [w2x2] [w3x3] ...] 단, w와x는 num_values차원의 벡터 -> w1x1 = C(scalar)
    return torch.sigmoid(torch.cat(w_out, dim = 1).sum(dim = 1))




In [47]:
class Poly2Model(nn.Module) : 
  def __init__(self, num_values) : 
    super().__init__()
    weights = []
    for (i, n1) in enumerate(num_values) : 
      w = []
      for (j, n2) in enumerate(num_values) : 
        if i < j : 
          w.append(nn.Embedding(n1 * n2, 1))
        else : 
          w.append(nn.Sequential())
      weights.append(nn.ModuleList(w))
    self.weights = nn.ModuleList(weights)
    self.num_values = num_values

  def forward(self,x) : 
    w_out = []
    for i in range(x.size(1)) : 
      for j in range(x.size(1)) : 
        if i < j : 
          w = self.weights[i][j](x[:,i] * self.num_values[j] + x[:,j])
          w_out.append(w)
    return torch.sigmoid(torch.cat(w_out, dim = 1).sum(dim = 1))

In [48]:
class FM(nn.Module) : 
  def __init__(self, num_values, dim) : 
    super().__init__()
    weights = []
    for n in num_values : 
      weights.append(nn.Embedding(n, dim))
      self.weights = nn.ModuleList(weights)
      self.dim = dim

  def forward(self,x) : 
    w_out = []
    for i in range(x.size(1)) : 
      for j in range(x.size(1)) : 
        if i < j :
          w1 = self.weights[i](x[:,i]).view(-1, 1, self.dim)
          w2 = self.weights[j](x[:,j]).view(-1, self.dim, 1)
          w = torch.bmm(w1, w2).view(-1, 1)
          w_out.append(w)
    return torch.sigmoid(torch.cat(w_out, dim = 1).sum(dim = 1))

In [49]:
class FFM(nn.Module) : 
  def __init__(self, num_values, dim) : 
    super().__init__()
    weights = []
    for n in num_values : 
      w = []
      for _ in range(len(num_values)) : 
        w.append(nn.Embedding(n, dim))
      weights.append(nn.ModuleList(w))
    self.weights = nn.ModuleList(weights)
    self.dim = dim

  def forward(self,x) : 
    w_out = []
    for i in range(x.size(1)) : 
      for j in range(x.size(1)) : 
        if i < j :
          w1 = self.weights[i][j](x[:,j]).view(-1, 1, self.dim)
          w2 = self.weights[j][i](x[:,j]).view(-1, self.dim, 1 )
          w = torch.bmm(w1, w2).view(-1, 1)
          w_out.append(w)

    return torch.sigmoid(torch.cat(w_out, dim = 1).sum(dim = 1))

In [50]:
def to_corrects(preds, labels) : 
  return ((preds - 0.5).ceil() == labels).float().numpy()

In [51]:
from torch import optim

epochs = 10
dim = 8

model = LinearModel(num_values)
loss_func = nn.BCELoss()
optimizer = optim.Adam(model.parameters())

In [52]:
for e in range(epochs) : 
  model.train()
  losses, corrects = [], []
  for x, y in trn_loader : 
    y_pred = model(x)
    loss = loss_func(y_pred, y.float())
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    losses.append(loss.item())
    corrects.append(to_corrects(y_pred, y))
  trn_loss = np.mean(losses)
  trn_acc = np.mean(np.concatenate(corrects))

  model.eval()
  losses, corrects = [], []
  for x, y in test_loader : 
    y_pred = model(x)
    losses.append(loss.item())
    corrects.append(to_corrects(y_pred, y))
  test_loss = np.mean(losses)
  test_acc = np.mean(np.concatenate(corrects))
